In [1]:
# Updated to include all SOTF2020

import pandas as pd
import numpy as np
import os
import datetime
from haversine import haversine
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/BR_Margaret_zips_SOTF2020_20200207'

In [2]:
df_SOTF_2020=pd.read_excel("./2020 SOTF Ad & Display Schedules.xlsx",
                           dtype=str,skiprows=1,sheet_name="2020 SOTF TEXT AD SCHEDULE")
df_SOTF_2020=df_SOTF_2020.rename(columns={"STORE #":"location_id"})
df_SOTF_2020.shape

(256, 11)

In [3]:
TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [4]:
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)

In [5]:
df_zip_by_store['zip_cd'].nunique()

12980

In [6]:
df_DMA_mapping=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",
                             skiprows=1,dtype=str)
df_DMA_mapping=df_DMA_mapping.iloc[:,[0,2]]
df_DMA_mapping.columns=['zip_cd','DMA']
print(df_DMA_mapping.shape)
df_DMA_mapping=df_DMA_mapping.drop_duplicates()
print(df_DMA_mapping.shape)


(51324, 2)
(43102, 2)


In [7]:
df_store_location=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200201-134613-934.txt",
                             sep="|",dtype=str)
df_store_location=df_store_location[['location_id','zip_cd','latitude_meas','longitude_meas']]
df_store_location['zip_cd']=df_store_location['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_location['latitude_meas']=df_store_location['latitude_meas'].astype(float)
df_store_location['longitude_meas']=df_store_location['longitude_meas'].astype(float)

df_store_lat_lng=df_store_location.loc[:,['location_id','zip_cd','latitude_meas','longitude_meas']]
df_store_lat_lng=df_store_lat_lng.rename(columns={"zip_cd":"store_zip"})

In [8]:
df_store_lat_lng.head(2)

,location_id,store_zip,latitude_meas,longitude_meas
0,3,30906,33.421570,-82.018921
1,30,43402,41.363132,-83.650909


In [9]:
df_SOTF_2020=pd.merge(df_SOTF_2020,df_store_lat_lng,on="location_id",how="left")

In [10]:
df_SOTF_2020_new=df_SOTF_2020[pd.isnull(df_SOTF_2020['latitude_meas'])]
df_SOTF_2020_existing=df_SOTF_2020[pd.notnull(df_SOTF_2020['latitude_meas'])]

In [11]:
list_store_not_found=['4721','5414','5421'] # received in email from Margaret
# RE: 2020 SOTF/Grand Openings - Local TV/Radio/Remotes Estimated Cost Request
# 20200218 at 12:52 PM

dict_new_stores={}

# 4710 	255 WEDDINGTON BRANCH ROAD	MAGIC MART SC	PIKEVILLE	KY	41501-3205
dict_new_stores.update({"4710":{"Google_Address":"255 Weddington Branch Rd, Pikeville, KY 41501","Google_Location":[37.5257758,-82.5685399]}})

# 4712 	2685 HILLTOP DRIVE	KMART SC	REDDING EPA ID, BAG BAN	CA	96002-1020
dict_new_stores.update({"4712":{"Google_Address":"2685 Hilltop Dr, Redding, CA 96002","Google_Location":[40.5684416,-122.357451]}})

# 4713 783 DELSEA DRIVE	COLLEGETOWN SHOPPING CENTER	GLASSBORO	NJ	08028
dict_new_stores.update({"4713":{"Google_Address":"731 Delsea Dr, Glassboro, NJ 08028","Google_Location":[39.72152,-75.116531]}})

# 4714 1251 STATE ROUTE 29	KMART PLAZA	GREENWICH, BAG BAN	NY	12834-6119
dict_new_stores.update({"4714":{"Google_Address":"1251 NY-29, Greenwich, NY 12834","Google_Location":[43.0973489,-73.5079692]}})

# 4715 4525 N ORACLE ROAD	TOYS R US CENTER	TUSCON	AZ	85705-1637
dict_new_stores.update({"4715":{"Google_Address":"4525 N Oracle Rd, Tucson, AZ 85705","Google_Location":[32.2885652,-110.9800392]}})

# 4716 4208 E BLUE GRASS RD	INDIAN HILLS PLAZA	MOUNT PLEASANT	MI	48858-7966
dict_new_stores.update({"4716":{"Google_Address":"4208 E Blue Grass Rd, Mt Pleasant, MI 48858","Google_Location":[43.5739459,-84.7649652]}})

# 4717 360 EAST 10TH ST	GILROY VILLAGE	GILROY EPA ID, BAG BAN	CA	95020-6576
dict_new_stores.update({"4717":{"Google_Address":"360 E 10th St, Gilroy, CA 95020","Google_Location":[37.0000558,-121.561048]}})

# 4718 1631 WEST CRAIG ROAD	NORTH MESA PLAZA	NORTH LAS VEGAS	NV	89032
dict_new_stores.update({"4718":{"Google_Address":"1631 W Craig Rd, North Las Vegas, NV 89030","Google_Location":[36.2381894,-115.1644729]}})

# 4719 3779 ROME ROAD	PULASKI PLAZA	PULASKI  BAG BAN	NY	13142-2402
dict_new_stores.update({"4719":{"Google_Address":"3779 NY-13 B, Pulaski, NY 13142","Google_Location":[43.5551558,-76.1163317]}})

# 4720 2700 US HIGHWAY 22	UNION 22 PLAZA	UNION	NJ	07083-8564
dict_new_stores.update({"4720":{"Google_Address":"2700 US-22, Union, NJ 07083","Google_Location":[40.6866464,-74.311087]}})

# 4721 400 Main Street, Dennis Port, MA, 02639
dict_new_stores.update({"4721":{"Google_Address":"400 Main St, Dennis Port, MA 02639","Google_Location":[41.66689,-70.1329873]}})

# 5405 1300 US HIGHWAY 127	FRANKLIN SQUARE & FRANKLIN WEST SHOPPING CENTER	FRANKFORT	KY	40601-4395
dict_new_stores.update({"5405":{"Google_Address":"1300 US-127, Frankfort, KY 40601","Google_Location":[38.166582,-84.8998445]}})

# 5412 2120 EAST DOROTHY LANE	KETTERING TOWN CENTER	KETTERING	OH	45420
dict_new_stores.update({"5412":{"Google_Address":"2120 E Dorothy Ln, Kettering, OH 45420","Google_Location":[39.701601,-84.1264112]}})

# 5413 340 SEVILLE ST	KOHLS SC	FLORENCE	AL	35634-1511
dict_new_stores.update({"5413":{"Google_Address":"350 Seville St, Florence, AL 35630","Google_Location":[34.8382162,-87.6269289]}})

# 5414 4125 Highway 20, Buford, GA, 30518
dict_new_stores.update({"5414":{"Google_Address":"4125 GA-20, Buford, GA 30518","Google_Location":[34.0935686,-84.0006123]}})

# 5415 7743 SUDLEY ROAD	SUDLEY TOWNE PLAZA	MANASSAS	VA	20109-2813
dict_new_stores.update({"5415":{"Google_Address":"7665 Sudley Rd, Manassas, VA 20109","Google_Location":[38.7900885,-77.5121781]}})

# 5417 558 BATTLEFIELD PARKWAY	FORT OGLETHORPE MARKETPLACE	FORT OGLERTHORPE	GA	30742-3849
dict_new_stores.update({"5417":{"Google_Address":"558 Battlefield Pkwy, Fort Oglethorpe, GA 30742","Google_Location":[34.9547068,-85.2557537]}})

# 5418 1550 COSHOCTON AVE	KNOX VILLAGE SQUARE	MOUNT VERNON	OH	43050-1475
dict_new_stores.update({"5418":{"Google_Address":"1500 Coshocton Ave (US Highway 36, Mt Vernon, OH 43050","Google_Location":[40.4001295,-82.4460567]}})

# 5419 1077 EAST STUART DRIVE SUITE 100	TWIN COUNTRY PLAZA	GALAX	VA	24333-2638
dict_new_stores.update({"5419":{"Google_Address":"1077 E Stuart Dr #100, Galax, VA 24333","Google_Location":[36.6845984,-80.8871837]}})

# 5420 4025 POPLAR LEVEL ROAD	KMART CENTER	LOUISVILLE	KY	40213-1523
dict_new_stores.update({"5420":{"Google_Address":"4025 Poplar Level Rd, Louisville, KY 40213","Google_Location":[38.2009162,-85.7071697]}})

# 5421 736 Warrenton Road, Fredericksburg, VA, 22406
dict_new_stores.update({"5421":{"Google_Address":"736 Warrenton Rd, Fredericksburg, VA 22406","Google_Location":[38.3531101,-77.5043113]}})


In [12]:
df_SOTF_2020_new['store_zip']=df_SOTF_2020_new['location_id'].apply(lambda x: dict_new_stores[x]['Google_Address'][-5:])
df_SOTF_2020_new['latitude_meas']=df_SOTF_2020_new['location_id'].apply(lambda x: dict_new_stores[x]['Google_Location'][0])
df_SOTF_2020_new['longitude_meas']=df_SOTF_2020_new['location_id'].apply(lambda x: dict_new_stores[x]['Google_Location'][1])

df_SOTF_2020=df_SOTF_2020_new.append(df_SOTF_2020_existing)
df_SOTF_2020=df_SOTF_2020.sort_values("location_id")

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [13]:
df_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200201-134613-934.txt",
                           dtype=str,sep="|",usecols=['location_id','zip_cd','latitude_meas','longitude_meas'])
df_store_list=df_store_list[['location_id','zip_cd','latitude_meas','longitude_meas']]
df_store_list['zip_cd']=df_store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
df_store_list['latitude_meas']=df_store_list['latitude_meas'].astype(float)
df_store_list['longitude_meas']=df_store_list['longitude_meas'].astype(float)
df_store_list=df_store_list[df_store_list['location_id']!="6990"]
df_store_list=df_store_list[df_store_list['location_id']!="145"]
df_store_list=df_store_list.rename(columns={"zip_cd":"store_zip"})
df_store_list.shape

(1404, 4)

In [14]:
df_SOTF_2020['SOTF_label']="SOTF_2020"
df_store_list['SOTF_label']="others"
df_store_list=df_SOTF_2020[df_store_list.columns.tolist()].append(df_store_list).drop_duplicates("location_id")

In [15]:
df_store_list.shape

(1425, 5)

In [16]:
df_store_list[df_store_list['SOTF_label']=="SOTF_2020"].shape


(256, 5)

In [17]:
df_zip_by_store.head(2)

,location_id,zip_cd,zip_type
0,1949,75040,P
1,1949,75098,P


In [18]:
TA_zips_for_all_stores=pd.DataFrame()

TA_zips_for_all_stores_1=pd.merge(df_store_list,df_zip_by_store,on="location_id",how="left")
TA_zips_for_all_stores_1.head(2)

,location_id,store_zip,latitude_meas,longitude_meas,SOTF_label,zip_cd,zip_type
0,1003,30120,34.175447,-84.787971,SOTF_2020,30120,P
1,1003,30120,34.175447,-84.787971,SOTF_2020,30121,P


In [19]:
TA_zips_for_all_stores_2=TA_zips_for_all_stores_1[pd.isnull(TA_zips_for_all_stores_1['zip_cd'])]
TA_zips_for_all_stores_1=TA_zips_for_all_stores_1[pd.notnull(TA_zips_for_all_stores_1['zip_cd'])]
print(TA_zips_for_all_stores_2.shape)

del TA_zips_for_all_stores_2['zip_cd']
del TA_zips_for_all_stores_2['zip_type']

(51, 7)


In [20]:
TA_zips_for_all_stores_2[pd.isnull(TA_zips_for_all_stores_2['latitude_meas'])].shape

(0, 5)

In [21]:
df_10_zips_all_new_stores=pd.DataFrame()
for ind, row in TA_zips_for_all_stores_2.iterrows():
    location_id=row['location_id']
    store_location=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_location,v,unit="mi")
        if dist<=10:
            # print(location_id,zip_cd)
            df=pd.DataFrame({"location_id":[location_id],"zip_cd":[zip_cd],"dist":[dist]})
            df_10_zips_all_new_stores=df_10_zips_all_new_stores.append(df)
df_10_zips_all_new_stores['zip_type']="zip_10"

In [22]:
print(df_10_zips_all_new_stores.shape,df_10_zips_all_new_stores['location_id'].nunique())

(1250, 4) 51


In [23]:
df_new_store_zips=df_10_zips_all_new_stores[['location_id','zip_cd','zip_type']]
TA_zips_for_all_stores_2=pd.merge(TA_zips_for_all_stores_2,df_new_store_zips,how="left")
TA_zips_for_all_stores=TA_zips_for_all_stores_1.append(TA_zips_for_all_stores_2)
print(TA_zips_for_all_stores.shape,TA_zips_for_all_stores['location_id'].nunique(),TA_zips_for_all_stores['zip_cd'].nunique())

(23846, 7) 1425 13455


In [24]:
TA_zips_for_all_stores_1['zip_type'].nunique()

3

In [25]:
TA_zips_for_all_stores['SOTF_label'].unique()

array(['SOTF_2020', 'others'], dtype=object)

In [26]:
df_output_1=TA_zips_for_all_stores[TA_zips_for_all_stores['SOTF_label']=="SOTF_2020"]
df_output_1.shape

(4405, 7)

In [27]:
TA_zips_for_all_stores.head(2)

,location_id,store_zip,latitude_meas,longitude_meas,SOTF_label,zip_cd,zip_type
0,1003,30120,34.175447,-84.787971,SOTF_2020,30120,P
1,1003,30120,34.175447,-84.787971,SOTF_2020,30121,P


In [28]:
TA_zips_for_all_stores.shape

(23846, 7)

In [29]:
df_output_2=TA_zips_for_all_stores.copy()
df_DMA_mapping_store=df_DMA_mapping.rename(columns={"zip_cd":"store_zip"})
df_output_2=pd.merge(df_output_2,df_DMA_mapping_store,on="store_zip",how="left").rename(columns={"DMA":"DMA_of_store"})
df_output_2=pd.merge(df_output_2,df_DMA_mapping,on="zip_cd",how="left").rename(columns={"DMA":"DMA_of_zip"})
print(df_output_2.shape)

(25577, 9)


In [30]:
df_output_2['SOTF_label'].unique()

array(['SOTF_2020', 'others'], dtype=object)

In [31]:
df_output_2_all=df_output_2.groupby("DMA_of_store")['location_id','zip_cd'].nunique().reset_index()
df_output_2_all=df_output_2_all.rename(columns={"location_id":"all_store_count","zip_cd":"all_TA_zip_count"})

df_output_2_SOTF=df_output_2[df_output_2['SOTF_label']=="SOTF_2020"]
df_output_2_SOTF=df_output_2_SOTF.groupby("DMA_of_store")['location_id','zip_cd'].nunique().reset_index()
df_output_2_SOTF=df_output_2_SOTF.rename(columns={"location_id":"SOTF2020_store_count","zip_cd":"SOTF2020_TA_zip_count"})

df_output_2=pd.merge(df_output_2_all,df_output_2_SOTF,on="DMA_of_store",how="outer")
df_output_2=df_output_2[['DMA_of_store','all_store_count','SOTF2020_store_count','all_TA_zip_count','SOTF2020_TA_zip_count']]
df_output_2=df_output_2.fillna(0)
df_output_2['Zip_Pctg_SOTF2020']=np.round(df_output_2['SOTF2020_TA_zip_count']/df_output_2['all_TA_zip_count'],4)



In [32]:
writer=pd.ExcelWriter("./BL_zips_by_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output_1.to_excel(writer,"request_1_zips_for_SOTF",index=False)

df_output_2.to_excel(writer,"request_2_by_DMA",index=False)

writer.save()

In [33]:
# Get the list of SOTF in multiple DMAs
df_output_1_qc=pd.merge(df_output_1,df_DMA_mapping_store,on="store_zip",how="left")
df_output_1_qc=df_output_1_qc.groupby("location_id")['DMA'].apply(set).to_frame().reset_index()
df_output_1_qc['DMA_count']=df_output_1_qc['DMA'].apply(len)
df_output_1_qc=df_output_1_qc.sort_values(["DMA_count",'location_id'],ascending=[False,True])
df_output_1_qc.shape

df_output_1_qc.to_csv("./BL_SOTF_in_multiple_DMAs_JL_"+str(datetime.datetime.now().date())+".csv",index=False)